# Data Input and Output

In this notebook:
* Reading flat files using Tidy Tools
* Working with external databases

# Overview

Preferably, use the Tidyverse packages:
* `readr` for flat files (CSV, TSV, FWF)
* `haven` for SPSS, State, SAS
* `readxl` for reading Excel files

Unless:
* you absolutely need best performance, then use `fread` in `data.table` for fast reading of flat files

## Note
Ideally, if you work with external files, then you request and share files in a common format, like CSV. It's best if, for example, SPSS itself writes a CSV file that you then use, as opposed to reading in a native SPSS file into R. Of course, that is not always possible, and so we have a choice of packages to use in such situations.

# Examples

In [ ]:
#library(readr)
#library(haven)
#library(readxl)

Usage:
```
df <- read.csv("file.csv")          # base R has "." in function name
df <- read.delim("file.txt")

df <- read_csv("file.csv")          # readr has "_" in function name
df <- read_delim("file.txt")

df <- read_sas("file.sas7bcat")     # haven
df <- read_sav("file.sav")
df <- read_dat("file.dta")

df <- read_excel("file.xlsx")       # readxl
```

# Comparison

### base R
Disadvantages:
* stringsAsFactors = TRUE 
* slow

Setting the first to FALSE globally is not good practice, because your code will not be reproducible. Always be explicit with this parameter.

Functions for reading data:
* read.table or read.csv and similar
* read.csv, read.csv2 for different delimiters
* read.delim, read.delim2 for different decimal symbol

Common options to play with:

```
df  <- read.csv(file="file.csv", header=TRUE, sep=",",
                stringsAsFactors=FALSE, row.names=1)
```

`row.names=1` is helpful, eg, when working with Python `pandas` output.

See `?read.table` for details

### Readr

Readr functions are similar to base R, with `_` replacing `.` :

Advantages:
* faster
* more intelligent
* characters are not converted to factors
* outputs `tbl_df`
* run `problems(df)` to see issues with data
* use `locale()` to specify options that can vary between languages and localities


Usage:
`df <- read_csv("file.csv", locale = locale())`

Details are here:  
* https://github.com/hadley/readr
* locale(): https://cran.r-project.org/web/packages/readr/vignettes/locales.html



## readxl
https://github.com/hadley/readxl

advantages:
* reads both `xls` and `xlsx`
* you can intuitively specify which sheets to read

### haven
https://github.com/hadley/haven

[Vignette on semantics (working)](https://github.com/hadley/haven/blob/master/vignettes/semantics.Rmd)


    df <- read_sas("file.sas7bcat")    # SAS
    df <- read_sav("file.sav")         # SPSS
    df <- read_dat("file.dta")         # Stata

## Example: reading in Excel data

A good practice when working with external data is to inspect it in a Unix terminal with `head` and `tail`, but this is usually not possible with proprietary formats. In that case, it's imperative to carefully check the data for any corruptions after loading it into R.

In [ ]:
library(dplyr)
library(readxl)

In [ ]:
# First, you may open the file in Excel to see what it looks like
# Here, we skip the first 4 rows in order to extract only the data table
summary_excel <- read_excel("small_data/Medicare_Charge_Inpatient_DRGALL_DRG_Summary_Reports_FY2014.xlsx",
                            col_names=TRUE,            # true by default
                            col_types=NULL,            # guess based on data (default) 
                            sheet="Averages_by_DRG",   # could be an integer
                            na="",                     # change as needed
                            skip=4) 

summary_excel %>% names

# Working with databases

Next, we explain how to use `dplyr` with a database. Here are two use cases:
* you already have data in a database
* your data does not fit in memory, so you have to access it from a database

Dplyr supports the three most popular open source databases (`sqlite`, `mysql` and `Postgresql`), and Google’s `bigquery`.

In [ ]:
library(DBI)

## Databases with dplyr
Requires no additional packages besides `dplyr`. You can use the same verbs like `select()`, `filter()`, etc.

    When working with databases, dplyr tries to be as lazy as possible:

    * It never pulls data into R unless you explicitly ask for it.
    * It delays doing any work until the last possible moment: it collects together everything you want to do and then sends it to the database in one step.

### Sqlite

```{r}
# create a database
my_db <- src_sqlite("DATABASE_NAME.sqlite3",
                    create = TRUE)  # ok to create new tables

# write DF to my_db
# "Because all the data has to flow through R,
# you should note that this is not suitable for very large datasets."
copy_to(my_db, MYTABLE)

res <- tbl(my_db, sql("SELECT * FROM DATABASE_NAME"))

collect(res)
```

### Postgres

```{r}
flights_postgres <- tbl(src_postgres("DATABASE_NAME"), "MYTABLE")
# and continue with regular dplyr syntax on `flights_postgres`
```

For more details, see:
* [SQL translation](https://cran.r-project.org/web/packages/dplyr/vignettes/databases.html#sql-translation)
* [Grouping](https://cran.r-project.org/web/packages/dplyr/vignettes/databases.html#grouping)
* [Bigquery](https://cran.r-project.org/web/packages/dplyr/vignettes/databases.html#bigquery)
* [Hadley's advice on picking a database](https://cran.r-project.org/web/packages/dplyr/vignettes/databases.html#picking-a-database)

# Other options

### 1 of 2: RPostgreSQL

```{r}

library(RPostgreSQL)

# open connection
driver <- dbDriver("PostgreSQL")

con <- dbConnect(drv, dbname = "DATABASE_NAME",
                 host = "localhost",
                 port = 5432,         # or any other port specified by your DBA
                 user = "USERNAME",
                 password = "PASSWORD")

# table exists?
dbExistsTable(con, "MYTABLE")

# write to database
dbWriteTable(con, "MYTABLE", 
             value = DF,        # your R dataframe
             append = TRUE,
             row.names = FALSE)
 
# query data 
res <- dbGetQuery(con, "SELECT * from MYTABLE")

# close connection
dbDisconnect(con)
dbUnloadDriver(drv)
```

### 2 of 2: RPostgres

```{r}
# seems to have performance advantages:
# https://github.com/rstats-db/RPostgres

# devtools::install_github("rstats-db/RPostgres")


# open connection
con <- dbConnect(RPostgres::Postgres(),
                 dbname="DATABASE_NAME")

# connect to a specific postgres database e.g. Heroku
con <- dbConnect(RPostgres::Postgres(),
                 dbname = 'DATABASE_NAME', 
                 host = 'HOST',    # i.e. 'ec2-54-83-201-96.compute-1.amazonaws.com'
                 port = 5432,      # or any other port specified by your DBA
                 user = 'USERNAME',
                 password = 'PASSWORD')

# list tables
dbListTables(con)

# write to database
dbWriteTable(conn=con,
             name="MYTABLE",
             value=DF,          # your R dataframe
             overwrite=TRUE)    # overwrite existing?


# print table
dbReadTable(con, "MYTABLE") %>% head

# query database
res <- dbSendQuery(con, "SELECT * FROM mtcars WHERE cyl = 4")
dbFetch(res)
dbClearResult(res)

# close connection
dbDisconnect(con)
```

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*